In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 16
tree_depth = 8
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.131396293640137 | KNN Loss: 6.227344036102295 | BCE Loss: 1.9040518999099731
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.219286918640137 | KNN Loss: 6.227286338806152 | BCE Loss: 1.9920002222061157
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.189247131347656 | KNN Loss: 6.226543426513672 | BCE Loss: 1.9627039432525635
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.169183731079102 | KNN Loss: 6.226537704467773 | BCE Loss: 1.9426462650299072
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.15272331237793 | KNN Loss: 6.2260355949401855 | BCE Loss: 1.926687479019165
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.198753356933594 | KNN Loss: 6.225461959838867 | BCE Loss: 1.973291039466858
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.116584777832031 | KNN Loss: 6.225305557250977 | BCE Loss: 1.8912787437438965
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.173934936523438 | KNN Loss: 6.224647521972656 | BCE Loss: 1.9492878

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 5.735421180725098 | KNN Loss: 4.626675605773926 | BCE Loss: 1.1087453365325928
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.6172637939453125 | KNN Loss: 4.494812965393066 | BCE Loss: 1.122450590133667
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.558211326599121 | KNN Loss: 4.4204607009887695 | BCE Loss: 1.1377503871917725
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 5.433544158935547 | KNN Loss: 4.292762279510498 | BCE Loss: 1.140782117843628
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.3397369384765625 | KNN Loss: 4.232474327087402 | BCE Loss: 1.1072626113891602
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 5.2252984046936035 | KNN Loss: 4.119917392730713 | BCE Loss: 1.1053811311721802
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.177340507507324 | KNN Loss: 4.050661563873291 | BCE Loss: 1.126678705215454
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.075552940368652 | KNN Loss: 3.9711804389953613 | BCE Lo

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.31995964050293 | KNN Loss: 3.2258055210113525 | BCE Loss: 1.094153881072998
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.285663604736328 | KNN Loss: 3.239567518234253 | BCE Loss: 1.0460963249206543
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.283781051635742 | KNN Loss: 3.248910903930664 | BCE Loss: 1.034869909286499
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.315661907196045 | KNN Loss: 3.242034912109375 | BCE Loss: 1.0736271142959595
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.284772872924805 | KNN Loss: 3.226449728012085 | BCE Loss: 1.0583233833312988
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.303432464599609 | KNN Loss: 3.245673418045044 | BCE Loss: 1.0577588081359863
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.270933628082275 | KNN Loss: 3.180241346359253 | BCE Loss: 1.0906922817230225
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 4.238199234008789 | KNN Loss: 3.1943888664245605 | BCE Loss

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.13721227645874 | KNN Loss: 3.1251296997070312 | BCE Loss: 1.012082576751709
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.149568557739258 | KNN Loss: 3.1208372116088867 | BCE Loss: 1.0287314653396606
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.170516014099121 | KNN Loss: 3.142583131790161 | BCE Loss: 1.0279330015182495
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.163435935974121 | KNN Loss: 3.113342761993408 | BCE Loss: 1.0500929355621338
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.1312336921691895 | KNN Loss: 3.119642496109009 | BCE Loss: 1.0115911960601807
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.195626258850098 | KNN Loss: 3.148836612701416 | BCE Loss: 1.0467896461486816
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.19155216217041 | KNN Loss: 3.149794816970825 | BCE Loss: 1.041757583618164
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 4.164190292358398 | KNN Loss: 3.1522083282470703 | BCE Los

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.186305999755859 | KNN Loss: 3.139052391052246 | BCE Loss: 1.0472533702850342
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.175289154052734 | KNN Loss: 3.1487531661987305 | BCE Loss: 1.0265361070632935
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.121947288513184 | KNN Loss: 3.1192805767059326 | BCE Loss: 1.00266695022583
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.216286659240723 | KNN Loss: 3.1491591930389404 | BCE Loss: 1.0671277046203613
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.175373077392578 | KNN Loss: 3.1491129398345947 | BCE Loss: 1.0262600183486938
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.200582981109619 | KNN Loss: 3.1648108959198 | BCE Loss: 1.0357720851898193
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.13418436050415 | KNN Loss: 3.1326236724853516 | BCE Loss: 1.0015605688095093
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 4.181057929992676 | KNN Loss: 3.1439993381500244 | BCE Lo

...

Epoch 448 / 500 | iteration 15 / 30 | Total Loss: 4.089088439941406 | KNN Loss: 3.071681499481201 | BCE Loss: 1.0174068212509155
Epoch 448 / 500 | iteration 20 / 30 | Total Loss: 4.125425338745117 | KNN Loss: 3.0975501537323 | BCE Loss: 1.027875304222107
Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 4.044632434844971 | KNN Loss: 3.066856861114502 | BCE Loss: 0.9777754545211792
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.0968170166015625 | KNN Loss: 3.0623044967651367 | BCE Loss: 1.0345127582550049
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.073984146118164 | KNN Loss: 3.0623459815979004 | BCE Loss: 1.0116382837295532
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 4.081967830657959 | KNN Loss: 3.0752947330474854 | BCE Loss: 1.0066732168197632
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 4.1162848472595215 | KNN Loss: 3.1017158031463623 | BCE Loss: 1.0145691633224487
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 4.095612049102783 | KNN Loss: 3.08211660385131

Epoch 459 / 500 | iteration 5 / 30 | Total Loss: 4.108442783355713 | KNN Loss: 3.052743434906006 | BCE Loss: 1.055699348449707
Epoch 459 / 500 | iteration 10 / 30 | Total Loss: 4.067770004272461 | KNN Loss: 3.056199073791504 | BCE Loss: 1.011570930480957
Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 4.112699508666992 | KNN Loss: 3.0618655681610107 | BCE Loss: 1.0508339405059814
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.136714458465576 | KNN Loss: 3.108010768890381 | BCE Loss: 1.0287036895751953
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.08895206451416 | KNN Loss: 3.0643649101257324 | BCE Loss: 1.0245869159698486
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 4.0924224853515625 | KNN Loss: 3.0653116703033447 | BCE Loss: 1.0271108150482178
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 4.111757755279541 | KNN Loss: 3.0552256107330322 | BCE Loss: 1.0565322637557983
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 4.106442451477051 | KNN Loss: 3.0806546211242676

Epoch 469 / 500 | iteration 25 / 30 | Total Loss: 4.0749053955078125 | KNN Loss: 3.053128242492676 | BCE Loss: 1.0217769145965576
Epoch   470: reducing learning rate of group 0 to 2.7058e-08.
Epoch 470 / 500 | iteration 0 / 30 | Total Loss: 4.154278755187988 | KNN Loss: 3.1213719844818115 | BCE Loss: 1.0329068899154663
Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 4.072127819061279 | KNN Loss: 3.065067768096924 | BCE Loss: 1.0070600509643555
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.122323036193848 | KNN Loss: 3.084078550338745 | BCE Loss: 1.038244605064392
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.089859962463379 | KNN Loss: 3.0588338375091553 | BCE Loss: 1.0310263633728027
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 4.087821006774902 | KNN Loss: 3.0655465126037598 | BCE Loss: 1.0222746133804321
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 4.069508075714111 | KNN Loss: 3.0296149253845215 | BCE Loss: 1.0398931503295898
Epoch 471 / 500 | iteration 0 / 3

Epoch 480 / 500 | iteration 15 / 30 | Total Loss: 4.045282363891602 | KNN Loss: 3.0295751094818115 | BCE Loss: 1.0157071352005005
Epoch 480 / 500 | iteration 20 / 30 | Total Loss: 4.100578308105469 | KNN Loss: 3.0946602821350098 | BCE Loss: 1.0059177875518799
Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 4.094522953033447 | KNN Loss: 3.0792617797851562 | BCE Loss: 1.015261173248291
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.060585021972656 | KNN Loss: 3.048348903656006 | BCE Loss: 1.0122363567352295
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.15877103805542 | KNN Loss: 3.103651523590088 | BCE Loss: 1.055119514465332
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 4.100108623504639 | KNN Loss: 3.0888283252716064 | BCE Loss: 1.0112801790237427
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 4.133885383605957 | KNN Loss: 3.100940227508545 | BCE Loss: 1.032944917678833
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 4.09637975692749 | KNN Loss: 3.0578696727752686 |

Epoch 491 / 500 | iteration 5 / 30 | Total Loss: 4.086661338806152 | KNN Loss: 3.0787205696105957 | BCE Loss: 1.0079410076141357
Epoch 491 / 500 | iteration 10 / 30 | Total Loss: 4.0501017570495605 | KNN Loss: 3.0588767528533936 | BCE Loss: 0.9912250638008118
Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 4.078587055206299 | KNN Loss: 3.0444648265838623 | BCE Loss: 1.0341222286224365
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.038388252258301 | KNN Loss: 3.0538175106048584 | BCE Loss: 0.9845705032348633
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.095339775085449 | KNN Loss: 3.0685949325561523 | BCE Loss: 1.0267446041107178
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 4.064320087432861 | KNN Loss: 3.0709352493286133 | BCE Loss: 0.993384838104248
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 4.153305530548096 | KNN Loss: 3.1162731647491455 | BCE Loss: 1.0370323657989502
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 4.1040754318237305 | KNN Loss: 3.0542881488

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 3.2370,  2.7679,  2.7122,  3.9908,  3.8847,  0.8725,  3.0327,  2.5618,
          2.2363,  1.6938,  2.2864,  2.5082,  0.6424,  2.1298,  1.3242,  1.0866,
          2.1310,  2.3357,  2.0970,  2.6638,  1.7034,  2.1554,  1.3457,  1.5651,
          2.0842,  1.5516,  1.5837,  1.6459,  1.2925,  0.4660, -0.3103,  0.5602,
          0.1078,  1.1122,  1.7155,  1.2236,  0.4354,  2.3027,  0.8398,  1.3784,
          0.6218, -0.6223, -0.1409,  2.6776,  2.5059,  0.5122, -0.3030,  0.2509,
          1.6245,  1.8960,  2.0517, -0.2963,  1.5548,  0.5244, -0.7659,  1.3300,
          1.7193,  1.1864,  1.4329,  1.3418,  0.5420,  0.9076, -0.0140,  1.9244,
          1.2947,  1.9178, -2.1642,  0.5483,  2.1487,  2.4974,  2.8048,  0.6341,
          1.2793,  2.8174,  1.6929,  0.9510, -0.0296,  0.4604,  0.0906,  1.9074,
         -0.1839,  0.4141,  0.8577, -0.2542,  0.4213, -1.1251, -2.7708, -0.2355,
          0.6990, -2.0052,  0.0347, -0.0140, -0.6624, -1.0185,  0.7804,  0.9752,
         -0.7505, -0.7863,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [12]:
dataset_ = [d[0].cpu() for d in dataset]

In [15]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 81.46it/s]


In [16]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [17]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [18]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [20]:
tensor_dataset = torch.stack(dataset_)

In [21]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [22]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [23]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [24]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [25]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [26]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [27]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [28]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [29]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [30]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [31]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [32]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [33]:
losses = []
accs = []
sparsity = []

In [34]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 018 | Total loss: 9.623 | Reg loss: 0.009 | Tree loss: 9.623 | Accuracy: 0.000000 | 0.241 sec/iter
Epoch: 00 | Batch: 001 / 018 | Total loss: 9.620 | Reg loss: 0.009 | Tree loss: 9.620 | Accuracy: 0.000000 | 0.229 sec/iter
Epoch: 00 | Batch: 002 / 018 | Total loss: 9.619 | Reg loss: 0.008 | Tree loss: 9.619 | Accuracy: 0.000000 | 0.227 sec/iter
Epoch: 00 | Batch: 003 / 018 | Total loss: 9.612 | Reg loss: 0.008 | Tree loss: 9.612 | Accuracy: 0.000000 | 0.225 sec/iter
Epoch: 00 | Batch: 004 / 018 | Total loss: 9.609 | Reg loss: 0.008 | Tree loss: 9.609 | Accuracy: 0.000000 | 0.227 sec/iter
Epoch: 00 | Batch: 005 / 018 | Total loss: 9.609 | Reg loss: 0.007 | Tree loss: 9.609 | Accuracy: 0.000000 | 0.226 sec/iter
Epoch: 00 | Batch: 006 / 018 | Total loss: 9.600 | Reg loss: 0.007 | Tree loss: 9.600 | Accuracy: 0.000000 | 0.226 sec/iter
Epoch: 00 | Batch

Epoch: 03 | Batch: 006 / 018 | Total loss: 9.526 | Reg loss: 0.006 | Tree loss: 9.526 | Accuracy: 0.058594 | 0.23 sec/iter
Epoch: 03 | Batch: 007 / 018 | Total loss: 9.529 | Reg loss: 0.006 | Tree loss: 9.529 | Accuracy: 0.062500 | 0.23 sec/iter
Epoch: 03 | Batch: 008 / 018 | Total loss: 9.515 | Reg loss: 0.006 | Tree loss: 9.515 | Accuracy: 0.076172 | 0.23 sec/iter
Epoch: 03 | Batch: 009 / 018 | Total loss: 9.517 | Reg loss: 0.006 | Tree loss: 9.517 | Accuracy: 0.080078 | 0.23 sec/iter
Epoch: 03 | Batch: 010 / 018 | Total loss: 9.521 | Reg loss: 0.006 | Tree loss: 9.521 | Accuracy: 0.050781 | 0.23 sec/iter
Epoch: 03 | Batch: 011 / 018 | Total loss: 9.517 | Reg loss: 0.007 | Tree loss: 9.517 | Accuracy: 0.060547 | 0.23 sec/iter
Epoch: 03 | Batch: 012 / 018 | Total loss: 9.517 | Reg loss: 0.007 | Tree loss: 9.517 | Accuracy: 0.068359 | 0.23 sec/iter
Epoch: 03 | Batch: 013 / 018 | Total loss: 9.510 | Reg loss: 0.007 | Tree loss: 9.510 | Accuracy: 0.074219 | 0.23 sec/iter
Epoch: 03 | Batc

Epoch: 06 | Batch: 013 / 018 | Total loss: 9.405 | Reg loss: 0.010 | Tree loss: 9.405 | Accuracy: 0.074219 | 0.228 sec/iter
Epoch: 06 | Batch: 014 / 018 | Total loss: 9.405 | Reg loss: 0.011 | Tree loss: 9.405 | Accuracy: 0.076172 | 0.228 sec/iter
Epoch: 06 | Batch: 015 / 018 | Total loss: 9.404 | Reg loss: 0.011 | Tree loss: 9.404 | Accuracy: 0.068359 | 0.228 sec/iter
Epoch: 06 | Batch: 016 / 018 | Total loss: 9.385 | Reg loss: 0.011 | Tree loss: 9.385 | Accuracy: 0.072266 | 0.228 sec/iter
Epoch: 06 | Batch: 017 / 018 | Total loss: 9.388 | Reg loss: 0.012 | Tree loss: 9.388 | Accuracy: 0.066253 | 0.228 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 07 | Batch: 000 / 018 | Total loss: 9.429 | Reg loss: 0.009 | Tree loss: 9.429 | Accuracy: 0.078125 | 0.228 sec/iter
Epoch: 07 | Batch: 001

Epoch: 10 | Batch: 000 / 018 | Total loss: 9.246 | Reg loss: 0.014 | Tree loss: 9.246 | Accuracy: 0.046875 | 0.229 sec/iter
Epoch: 10 | Batch: 001 / 018 | Total loss: 9.230 | Reg loss: 0.014 | Tree loss: 9.230 | Accuracy: 0.064453 | 0.229 sec/iter
Epoch: 10 | Batch: 002 / 018 | Total loss: 9.204 | Reg loss: 0.014 | Tree loss: 9.204 | Accuracy: 0.072266 | 0.229 sec/iter
Epoch: 10 | Batch: 003 / 018 | Total loss: 9.193 | Reg loss: 0.014 | Tree loss: 9.193 | Accuracy: 0.060547 | 0.229 sec/iter
Epoch: 10 | Batch: 004 / 018 | Total loss: 9.177 | Reg loss: 0.014 | Tree loss: 9.177 | Accuracy: 0.078125 | 0.229 sec/iter
Epoch: 10 | Batch: 005 / 018 | Total loss: 9.167 | Reg loss: 0.015 | Tree loss: 9.167 | Accuracy: 0.066406 | 0.229 sec/iter
Epoch: 10 | Batch: 006 / 018 | Total loss: 9.158 | Reg loss: 0.015 | Tree loss: 9.158 | Accuracy: 0.068359 | 0.229 sec/iter
Epoch: 10 | Batch: 007 / 018 | Total loss: 9.152 | Reg loss: 0.015 | Tree loss: 9.152 | Accuracy: 0.060547 | 0.229 sec/iter
Epoch: 1

Epoch: 13 | Batch: 007 / 018 | Total loss: 8.751 | Reg loss: 0.020 | Tree loss: 8.751 | Accuracy: 0.060547 | 0.229 sec/iter
Epoch: 13 | Batch: 008 / 018 | Total loss: 8.734 | Reg loss: 0.021 | Tree loss: 8.734 | Accuracy: 0.072266 | 0.229 sec/iter
Epoch: 13 | Batch: 009 / 018 | Total loss: 8.686 | Reg loss: 0.021 | Tree loss: 8.686 | Accuracy: 0.087891 | 0.229 sec/iter
Epoch: 13 | Batch: 010 / 018 | Total loss: 8.690 | Reg loss: 0.021 | Tree loss: 8.690 | Accuracy: 0.068359 | 0.229 sec/iter
Epoch: 13 | Batch: 011 / 018 | Total loss: 8.651 | Reg loss: 0.022 | Tree loss: 8.651 | Accuracy: 0.080078 | 0.229 sec/iter
Epoch: 13 | Batch: 012 / 018 | Total loss: 8.628 | Reg loss: 0.022 | Tree loss: 8.628 | Accuracy: 0.093750 | 0.229 sec/iter
Epoch: 13 | Batch: 013 / 018 | Total loss: 8.623 | Reg loss: 0.023 | Tree loss: 8.623 | Accuracy: 0.060547 | 0.229 sec/iter
Epoch: 13 | Batch: 014 / 018 | Total loss: 8.597 | Reg loss: 0.023 | Tree loss: 8.597 | Accuracy: 0.060547 | 0.229 sec/iter
Epoch: 1

...

Epoch: 83 | Batch: 016 / 018 | Total loss: 4.060 | Reg loss: 0.036 | Tree loss: 4.060 | Accuracy: 0.072266 | 0.231 sec/iter
Epoch: 83 | Batch: 017 / 018 | Total loss: 4.107 | Reg loss: 0.037 | Tree loss: 4.107 | Accuracy: 0.053830 | 0.231 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 84 | Batch: 000 / 018 | Total loss: 4.288 | Reg loss: 0.036 | Tree loss: 4.288 | Accuracy: 0.070312 | 0.231 sec/iter
Epoch: 84 | Batch: 001 / 018 | Total loss: 4.231 | Reg loss: 0.036 | Tree loss: 4.231 | Accuracy: 0.068359 | 0.231 sec/iter
Epoch: 84 | Batch: 002 / 018 | Total loss: 4.289 | Reg loss: 0.036 | Tree loss: 4.289 | Accuracy: 0.070312 | 0.231 sec/iter
Epoch: 84 | Batch: 003 / 018 | Total loss: 4.318 | Reg loss: 0.036 | Tree loss: 4.318 | Accuracy: 0.062500 | 0.231 sec/iter
Epoch: 84 | Batch: 004

Epoch: 87 | Batch: 003 / 018 | Total loss: 4.294 | Reg loss: 0.036 | Tree loss: 4.294 | Accuracy: 0.064453 | 0.231 sec/iter
Epoch: 87 | Batch: 004 / 018 | Total loss: 4.205 | Reg loss: 0.036 | Tree loss: 4.205 | Accuracy: 0.070312 | 0.231 sec/iter
Epoch: 87 | Batch: 005 / 018 | Total loss: 4.177 | Reg loss: 0.036 | Tree loss: 4.177 | Accuracy: 0.072266 | 0.231 sec/iter
Epoch: 87 | Batch: 006 / 018 | Total loss: 4.235 | Reg loss: 0.036 | Tree loss: 4.235 | Accuracy: 0.054688 | 0.231 sec/iter
Epoch: 87 | Batch: 007 / 018 | Total loss: 4.256 | Reg loss: 0.036 | Tree loss: 4.256 | Accuracy: 0.082031 | 0.231 sec/iter
Epoch: 87 | Batch: 008 / 018 | Total loss: 4.224 | Reg loss: 0.036 | Tree loss: 4.224 | Accuracy: 0.068359 | 0.231 sec/iter
Epoch: 87 | Batch: 009 / 018 | Total loss: 4.160 | Reg loss: 0.036 | Tree loss: 4.160 | Accuracy: 0.082031 | 0.231 sec/iter
Epoch: 87 | Batch: 010 / 018 | Total loss: 4.127 | Reg loss: 0.036 | Tree loss: 4.127 | Accuracy: 0.062500 | 0.231 sec/iter
Epoch: 8

Epoch: 90 | Batch: 010 / 018 | Total loss: 4.171 | Reg loss: 0.037 | Tree loss: 4.171 | Accuracy: 0.062500 | 0.231 sec/iter
Epoch: 90 | Batch: 011 / 018 | Total loss: 4.160 | Reg loss: 0.037 | Tree loss: 4.160 | Accuracy: 0.066406 | 0.231 sec/iter
Epoch: 90 | Batch: 012 / 018 | Total loss: 4.093 | Reg loss: 0.037 | Tree loss: 4.093 | Accuracy: 0.074219 | 0.231 sec/iter
Epoch: 90 | Batch: 013 / 018 | Total loss: 4.119 | Reg loss: 0.037 | Tree loss: 4.119 | Accuracy: 0.064453 | 0.231 sec/iter
Epoch: 90 | Batch: 014 / 018 | Total loss: 4.118 | Reg loss: 0.037 | Tree loss: 4.118 | Accuracy: 0.068359 | 0.231 sec/iter
Epoch: 90 | Batch: 015 / 018 | Total loss: 4.140 | Reg loss: 0.037 | Tree loss: 4.140 | Accuracy: 0.068359 | 0.231 sec/iter
Epoch: 90 | Batch: 016 / 018 | Total loss: 4.071 | Reg loss: 0.037 | Tree loss: 4.071 | Accuracy: 0.074219 | 0.231 sec/iter
Epoch: 90 | Batch: 017 / 018 | Total loss: 4.052 | Reg loss: 0.037 | Tree loss: 4.052 | Accuracy: 0.078675 | 0.231 sec/iter
Average 

Epoch: 93 | Batch: 017 / 018 | Total loss: 4.018 | Reg loss: 0.037 | Tree loss: 4.018 | Accuracy: 0.086957 | 0.231 sec/iter
Average sparseness: 0.9821428571428573
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
Epoch: 94 | Batch: 000 / 018 | Total loss: 4.304 | Reg loss: 0.037 | Tree loss: 4.304 | Accuracy: 0.058594 | 0.231 sec/iter
Epoch: 94 | Batch: 001 / 018 | Total loss: 4.269 | Reg loss: 0.037 | Tree loss: 4.269 | Accuracy: 0.060547 | 0.231 sec/iter
Epoch: 94 | Batch: 002 / 018 | Total loss: 4.254 | Reg loss: 0.037 | Tree loss: 4.254 | Accuracy: 0.072266 | 0.231 sec/iter
Epoch: 94 | Batch: 003 / 018 | Total loss: 4.203 | Reg loss: 0.037 | Tree loss: 4.203 | Accuracy: 0.066406 | 0.231 sec/iter
Epoch: 94 | Batch: 004 / 018 | Total loss: 4.224 | Reg loss: 0.037 | Tree loss: 4.224 | Accuracy: 0.066406 | 0.231 sec/iter
Epoch: 94 | Batch: 005

Epoch: 97 | Batch: 004 / 018 | Total loss: 4.211 | Reg loss: 0.037 | Tree loss: 4.211 | Accuracy: 0.068359 | 0.231 sec/iter
Epoch: 97 | Batch: 005 / 018 | Total loss: 4.241 | Reg loss: 0.037 | Tree loss: 4.241 | Accuracy: 0.078125 | 0.231 sec/iter
Epoch: 97 | Batch: 006 / 018 | Total loss: 4.253 | Reg loss: 0.037 | Tree loss: 4.253 | Accuracy: 0.064453 | 0.231 sec/iter
Epoch: 97 | Batch: 007 / 018 | Total loss: 4.167 | Reg loss: 0.037 | Tree loss: 4.167 | Accuracy: 0.076172 | 0.231 sec/iter
Epoch: 97 | Batch: 008 / 018 | Total loss: 4.138 | Reg loss: 0.037 | Tree loss: 4.138 | Accuracy: 0.070312 | 0.231 sec/iter
Epoch: 97 | Batch: 009 / 018 | Total loss: 4.107 | Reg loss: 0.037 | Tree loss: 4.107 | Accuracy: 0.087891 | 0.231 sec/iter
Epoch: 97 | Batch: 010 / 018 | Total loss: 4.153 | Reg loss: 0.037 | Tree loss: 4.153 | Accuracy: 0.068359 | 0.231 sec/iter
Epoch: 97 | Batch: 011 / 018 | Total loss: 4.116 | Reg loss: 0.037 | Tree loss: 4.116 | Accuracy: 0.078125 | 0.231 sec/iter
Epoch: 9

In [35]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [37]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 7.996078431372549


# Extract Rules

# Accumulate samples in the leaves

In [38]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 255


In [39]:
method = 'greedy'

In [40]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [44]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
#     print(conds)
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
6307
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Patte

============== Pattern 224 ==============
============== Pattern 225 ==============
============== Pattern 226 ==============
============== Pattern 227 ==============
============== Pattern 228 ==============
============== Pattern 229 ==============
============== Pattern 230 ==============
============== Pattern 231 ==============
============== Pattern 232 ==============
============== Pattern 233 ==============
============== Pattern 234 ==============
============== Pattern 235 ==============
============== Pattern 236 ==============
============== Pattern 237 ==============
============== Pattern 238 ==============
============== Pattern 239 ==============
============== Pattern 240 ==============
============== Pattern 241 ==============
============== Pattern 242 ==============
============== Pattern 243 ==============
============== Pattern 244 ==============
============== Pattern 245 ==============
============== Pattern 246 ==============
============== Pattern 247 =======